Importation des modules nécessaires. Puis upload des fichiers à traiter

In [ ]:
import pandas as pd
from google.colab import files

extraction = files.upload()


        

Définition des variables globales

In [ ]:
DATE1="2022-04-01"
DATE2="2022-04-30"

data = pd.read_excel("Extraction_1_PMA.xlsx", index_col="Ordre", parse_dates=True)

Renommage de certaines colonnes puis filtrage des ordres de travail selon les dates données au début

In [ ]:
data = data.rename(columns={'Date début plf': 'Date_début_plf' , 
                            'Poste technique': 'Departement', 
                            'Grpe gest. PM': 'Groupe_Gestionnaire',
                            'Statut util.' : 'Etat_OT'})


data = data[(data.Date_début_plf>=DATE1) & (data.Date_début_plf<=DATE2)]



Repérer les OT sans département.

In [ ]:
data["Departement"].fillna("XXXXXXXX", inplace = True)

In [ ]:
data = data.dropna(subset=['Departement','Etat_OT'])

Retrait des OT du département D car gérés par la MC. Puis renommage des départements selon leurs initiales.

In [ ]:
for i in data.index:
    if(data.loc[i,'Departement'][5]== ('D')):
        data.drop(i, axis=0,inplace=True)
    else:
        data.at[i,'Departement'] = data.loc[i,'Departement'][5]


Tableau croisé des OT selon leurs départements et leurs états.

In [ ]:
a = data.groupby(['Departement','Etat_OT']).Etat_OT.count()
tab_crois = a.unstack(fill_value=0)
tab_crois.loc['Total_Général_colonne']= tab_crois.sum(numeric_only=True, axis=0)
tab_crois.loc[:,'Total_Général_ligne'] = tab_crois.sum(numeric_only=True, axis=1)




Chercher les départements sans OT pour la période donnée puis les rajouter au compte afin de le noter sur le rapport final.(X étant les OT sans département)

In [ ]:
Etat_possible=["0","1","1_AA","1_AM","1_AP","1_AR","2","3","3_I","3_N","3_T"]
Etat_manquant = []
for element in Etat_possible:
    if element not in tab_crois:
        Etat_manquant.append(element)

Etat_manquant
for i in Etat_manquant:
    tab_crois[i]=0

Depart_possible=["E","M","P","S","T","X"]

Depart_manquant = []
for element in Depart_possible:
    if element not in tab_crois.index:
        Depart_manquant.append(element)

Depart_manquant

for i in Depart_manquant:
    tab_crois.loc[i]=0


tab_crois

Création d'un tableau récapitulatif donnant une synthèse plus détaillé de l'avancement des OT pour chaque département de l'usine.

In [ ]:
tab_recap=a.unstack(fill_value=0)

Etat_manquant = []
for element in Etat_possible:
    if element not in tab_recap:
        Etat_manquant.append(element)

Etat_manquant
for i in Etat_manquant:
    tab_recap[i]=0


Depart_manquant = []
for element in Depart_possible:
    if element not in tab_recap.index:
        Depart_manquant.append(element)

Depart_manquant

for i in Depart_manquant:
    tab_recap.loc[i]=0


tab_recap['1']+=tab_recap['1_AA']+tab_recap['1_AM']+tab_recap['1_AP']+tab_recap['1_AR']+tab_recap['2']+tab_recap['3_T']
tab_recap = tab_recap.drop(['1_AA','1_AM','1_AP','1_AR','2','3_T'],axis=1)

tab_recap.loc['Total_Général']= tab_recap.sum(numeric_only=True, axis=0)
tab_recap.loc[:,'Total_Général'] = tab_recap.sum(numeric_only=True, axis=1)

tab_recap.loc[:,'Nbr_OT_Prévues'] = tab_recap['Total_Général']-tab_recap['3_I']

tab_recap = tab_recap.drop(['Total_Général'],axis=1)
tab_recap = tab_recap.drop(['X'])

tab_recap.loc[:,"%Réalisation"] = round(tab_recap['3']/tab_recap['Nbr_OT_Prévues']*100,0)

tab_recap = tab_recap[["Nbr_OT_Prévues","3","0","1","3_I","3_N","%Réalisation"]]

tab_recap = tab_recap.rename(columns={'3': 'Soldé archivé' , 
                            '0': 'A faire', 
                            '1': 'En cours',
                            'Nbr_OT_Prévues' : 'Nbr OT Prévues'})

tab_recap = tab_recap.rename(index={'E': 'Emboutissage' , 
                            'T': 'Tôlerie', 
                            'P': 'Peinture',
                            'S': 'Sous-ensemble',
                            'M': 'Montage',
                            'Total_Général': 'USINE'})

tab_recap = tab_recap.T
tab_recap = tab_recap[['Emboutissage', 'Tôlerie', 'Peinture','Sous-ensemble','Montage','USINE']]
tab_recap = tab_recap.T
tab_recap.fillna(0, inplace = True)
tab_recap = tab_recap.astype('int64')
tab_recap 

Section consacré à la création d'un fichier Excel rassemblant toute la synthèse.

In [ ]:
import openpyxl

In [ ]:
writer = pd.ExcelWriter('Synthèse_PMA.xlsx')
tab_recap.to_excel(writer, sheet_name='tab_recap')
tab_crois.to_excel(writer, sheet_name='tab_crois')
writer.save()

Mise en forme du tableau récapitulatif selon le modèle donné.

In [ ]:
workbook = openpyxl.load_workbook('Synthèse_PMA.xlsx')
sheet = workbook['tab_recap']
sheet.insert_rows(0)
sheet.merge_cells('A1:H1')


cell_1_1=sheet.cell(1, 1)
cell_1_1.value = 'Avancement PMA entre '+DATE1+" et "+DATE2
cell_1_1.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'D6DCE4')

for row in sheet:
  for cell in row:
    cell.font = openpyxl.styles.Font(name = 'Calibri', size = 11, italic = False, bold = True, color = '000000')
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')

borderStyle1 = openpyxl.styles.Side(style = 'thin', color = '000000')
borderStyle2 = openpyxl.styles.Side(style = 'medium', color = '000000')
for row in sheet.iter_rows(min_row = 2, max_row = 8, min_col = 1, max_col = 8):
  for cell in row:
    cell.border = openpyxl.styles.Border(left = borderStyle1, right = borderStyle1, top = borderStyle1, bottom = borderStyle1)
    if cell.column==8:
      cell.border = openpyxl.styles.Border(left = borderStyle1, right = borderStyle2, top = borderStyle1, bottom = borderStyle1)
    if cell.row==8:
      cell.border = openpyxl.styles.Border(left = borderStyle1, right = borderStyle1, top = borderStyle1, bottom = borderStyle2)
    if cell.row==8 and cell.column==8:
      cell.border = openpyxl.styles.Border(left = borderStyle1, right = borderStyle2, top = borderStyle1, bottom = borderStyle2)

for i in range(1,9):
  cell=sheet.cell(2, i)
  cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'FFC000')
  cell=sheet.cell(1, i)
  cell.border = openpyxl.styles.Border(left = borderStyle2, right = borderStyle2, top = borderStyle2, bottom = borderStyle2)

for row in sheet.iter_rows(min_row = 3, max_row = 8, min_col = 8, max_col = 8):
  for cell in row:
    if cell.value <60:
      cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'FF0000')
    elif (cell.value >=60) and (cell.value < 90):
      cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'FF7F27')
    else:
      cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = '92D050')

sheet.column_dimensions['A'].width = 14
sheet.column_dimensions['B'].width = 15
sheet.column_dimensions['C'].width = 15.5
sheet.column_dimensions['H'].width = 12.5






Mise en forme du tableau croisé selon le modèle donné, puis sauvegarde du fichier final.

In [ ]:
sheet = workbook['tab_crois']

sheet.cell(8, 1).value = 'Sans département'

for row in sheet:
  for cell in row:
    cell.font = openpyxl.styles.Font(name = 'Calibri', size = 11, italic = False, bold = False, color = '000000')
    cell.alignment = openpyxl.styles.Alignment(horizontal = 'center')

for i in range(1,14):
  cell=sheet.cell(1, i)
  cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'D9E1F2')
  cell.font = openpyxl.styles.Font(name = 'Calibri', size = 11, italic = False, bold = True, color = '000000')
  cell=sheet.cell(7, i)
  cell.fill=openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'D9E1F2')
  cell.font = openpyxl.styles.Font(name = 'Calibri', size = 11, italic = False, bold = True, color = '000000')

sheet.column_dimensions['A'].width = 22
sheet.column_dimensions['M'].width = 19


workbook.save('Synthèse PMA finale.xlsx')